---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [5]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [109]:
import re

def date_sorter():
    pattern_month1 = '(?:(?:1|[2-9]{1}|1[0-2]{1})|(?:0[1-9]{1}|1[0-2]{1}))'
    pattern_month2 = '(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)'
    pattern_day = '(?:(?:0[1-9]{1}|1[0-9]{1}|2[0-9]{1}|3[0-1]{1})|(?:[1-9]{1}|1[0-9]{1}|2[0-9]{1}|3[0-1]{1}))'
    pattern_year1 = '(?:(?:19|20)?\d{2})'
    pattern_year2 = '(?:(?:19|20)\d{2})'
    
    regex1 = '(?:{}(?:\/|\-){}(?:\/|\-){})'.format(pattern_month1, pattern_day, pattern_year1)
    regex2 = '(?:{}\/{})'.format(pattern_month1, pattern_year2)
    regex3 = '(?:{}(?:\-|\ |\.\ ){}(?:st|nd|rd|th)?(?:\-|\,\ |\ ){})'.format(pattern_month2, pattern_day, pattern_year2)
    regex4 = '(?:{} {}(?:\ |\.\ |\,\ ){})'.format(pattern_day, pattern_month2, pattern_year2)
    regex5 = '(?:{}(?:|\ |\.\ |\,\ ){})'.format(pattern_month2, pattern_year2)
    regex6 = '(?:{})'.format(pattern_year2)

    memo = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    dates = list()
    for row in df:
        # Match pattern 1
        if len(re.findall(regex1, row)) != 0:
            date1 = re.findall(regex1, row)[0]
            date2 = re.split(r'\/|-', date1)
            if len(date2[2]) == 2:
                date2[2] = '19' + str(date2[2])
            date3 = '{}{:02}{:02}'.format(int(date2[2]), int(date2[0]), int(date2[1]))
            dates.append(date3)
            
        # Match pattern 2
        elif len(re.findall(regex2, row)) != 0:
            date1 = re.findall(regex2, row)[0]
            date2 = date1.split('/')
            date3 = '{}{:02}01'.format(int(date2[1]), int(date2[0]))
            dates.append(date3)
        
        # Match pattern 3
        elif len(re.findall(regex3, row)) != 0:
            date1 = re.findall(regex3, row)[0]
            date2 = re.split(r'\-|\ |\.\ |\,\ ', date1)
            month = memo[date2[0][0:3]]
            date3 = '{}{:02}{:02}'.format(int(date2[2]), month, int(date2[1]))
            dates.append(date3)
            
        # Match pattern 4
        elif len(re.findall(regex4, row)) != 0:
            date1 = re.findall(regex4, row)[0]
            date2 = re.split(r'\-|\ |\.\ |\,\ ', date1)
            month = memo[date2[1][0:3]]
            date3 = '{}{:02}{:02}'.format(int(date2[2]), month, int(date2[0]))
            dates.append(date3)
        
        # Match pattern 5
        elif len(re.findall(regex5, row)) != 0:
            date1 = re.findall(regex5, row)[0]
            date2 = re.split(r'\-|\ |\.\ |\,\ ', date1)
            month = memo[date2[0][0:3]]
            date3 = '{}{:02}01'.format(int(date2[1]), month)
            dates.append(date3)
        
        # Match pattern 6
        elif len(re.findall(regex6, row)) != 0:
            date1 = re.findall(regex6, row)[0]
            date2 = '{}0101'.format(int(date1))
            dates.append(date2)
    
    dates = pd.Series(dates)
    dates.sort_values(inplace=True)
    order = pd.Series(dates.index)
    
    return order